In [42]:
from itertools import combinations
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import numpy as np
import time
from datetime import datetime
import openpyxl
from sklearn.base import BaseEstimator
from sklearn.feature_selection import SelectorMixin
from sklearn.metrics import mean_absolute_error

Два класса для написания алгоритма. Указаны функции вычисляющие верхнюю и нижнюю границы для аппроксимации. 

In [18]:
class RoughSetsReducer:

    def __size(self, x):
        return (1, x.shape[0]) if x.ndim == 1 else x.shape

    '''
    Вычисляется отношение неразличимости
    '''
    def indisc(self, a, x):

        def codea(a, x, b):
            yy = 0
            for i in range(0, a):
                yy += x[i] * b**(a-(i+1))
            return yy

        p, q = self.__size(x)
        ap, aq = self.__size(a)
        z = [e for e in range(1, q+1)]
        tt = np.setdiff1d(z, a)
        tt_ind = np.setdiff1d(z, tt)-1
        if x.ndim == 1:
            x = x[tt_ind]
        else:
            x = x[:, tt_ind]
        y = x
        v = [codea(aq, y, 10) for i in range(0, p)] if y.ndim == 1 \
            else [codea(aq, y[i, :], 10) for i in range(0, p)]
        y = np.transpose(v)
        if y.shape[0] == 1 and len(y.shape) == 1:
            I, yy = [1], [y]
            y = np.hstack((y, I))
            b, k, l = [y], [1], [1]
        else:
            ax = 1 if y.ndim > 1 else 0
            yy = np.sort(y, axis=ax)
            I = y.argsort(axis=ax)
            y = np.hstack((yy, I))
            b, k, l = np.unique(yy, return_index=True, return_inverse=True)
        y = np.hstack((l, I))
        m = np.max(l)
        aa = np.zeros((m+1, p), dtype=int)
        for ii in range(0, m+1):
            for j in range(0, p):
                if l[j] == ii:
                    aa[ii, j] = I[j]+1
        return aa

    '''
    Вычисляется нижняя граница аппроксимации для y
    '''
    def rslower(self, y, a, T):
        z = self.indisc(a, T)
        w = []
        p, q = self.__size(z)
        for u in range(0, p):
            zz = np.setdiff1d(z[u, :], 0)
            if np.in1d(zz, y).all():
                w = np.hstack((w, zz))
        print("Вычисляется нихняя граница:")
        return w.astype(dtype=int)
        print(w.astype(dtype=int))

    '''
    Вычисляется верхняя граница аппроксимации для y
    '''
    def rsupper(self, y, a, T):
        z = self.indisc(a, T)
        w = []
        p, q = self.__size(z)
        for u in range(0, p):
            zz = np.setdiff1d(z[u, :], 0)
            zzz = np.intersect1d(zz, y)
            if len(zzz) > 0:
                w = np.hstack((w, zz))
        print("Вычисляется верхняя граница:")
        return w.astype(dtype=int)
        print(w.astype(dtype=int))


    def __pospq(self, p, q):
        pm, pn = self.__size(p)
        qm, qn = self.__size(q)
        num = 0
        pp, qq = [[]] * pm, [[]] * qm
        for i in range(0, pm):
            pp[i] = np.unique(p[i, :])
        for j in range(0, qm):
            qq[j] = np.unique(q[j, :])
        b = []
        for i in range(0, qm):
            for j in range(0, pm):
                if np.in1d(pp[j], qq[i]).all():
                    num += 1
                    b = np.hstack((b, pp[j]))
        bb = np.unique(b)
        if bb.size == 0:
            dd = 1
        else:
            _, dd = self.__size(bb)
        y = float(dd - 1)/pn if 0 in bb else float(dd)/pn
        b = np.setdiff1d(bb, 0)
        return y, b

    '''
    Перевод множеств C и D
    '''
    def core(self, C, D):
        x = np.hstack((C, D))
        c = np.array(range(1, C.shape[1]+1))
        d = np.array([C.shape[1]+1])
        cp, cq = self.__size(c)
        q = self.indisc(d, x)
        pp = self.indisc(c, x)
        b, w = self.__pospq(pp, q)
        a, k, kk, p = ([[]] * cq for i in range(4))
        y = []
        for u in range(0, cq):
            ind = u+1
            a[u] = np.setdiff1d(c, ind)
            p[u] = self.indisc(a[u], x)
            k[u], kk[u] = self.__pospq(p[u], q)
            if k[u] != b:
                y = np.hstack((y, ind))
        return np.array(y)

    def __sgf(self, a, r, d, x):
        pr = self.indisc(r, x)
        q = self.indisc(d, x)
        b = np.hstack((r, a))
        pb = self.indisc(b, x)
        p1, _ = self.__pospq(pb, q)
        p2, _ = self.__pospq(pr, q)
        return p1 - p2

    '''
    Уменьшение размерности
    '''
    def reduce(self, C, D):

        def redu2(i, re, c, d, x):
            yre = re
            re1, re2 = self.__size(re)
            q = self.indisc(d, x)
            p = self.indisc(c, x)
            pos_cd, _ = self.__pospq(p, q)
            y, j = None, None
            for qi in range(i, re2):
                re = np.setdiff1d(re, re[qi])
                red = self.indisc(re, x)
                pos_red, _ = self.__pospq(red, q)
                if np.array_equal(pos_cd, pos_red):
                    y = re
                    j = i
                    break
                else:
                    y = yre
                    j = i + 1
                    break
            return y, j

        x = np.hstack((C, D))
        c = np.array(range(1, C.shape[1]+1))
        d = np.array([C.shape[1]+1])
        y = self.core(C, D)
        q = self.indisc(d, x)
        p = self.indisc(c, x)
        pos_cd, _ = self.__pospq(p, q)
        re = y
        red = self.indisc(y, x)
        pos_red, _ = self.__pospq(red, q)
        while pos_cd != pos_red:
            cc = np.setdiff1d(c, re)
            c1, c2 = self.__size(cc)
            yy = [0] * c2
            for i in range(0, c2):
                yy[i] = self.__sgf(cc[i], re, d, x)
            cd = np.setdiff1d(c, y)
            d1, d2 = self.__size(cd)
            for i in range(d2, c2, -1):
                yy[i] = []
            ii = np.argsort(yy)
            for v1 in range(c2-1, -1, -1):
                v2 = ii[v1]
                re = np.hstack((re, cc[v2]))
                red = self.indisc(re, x)
                pos_red, _ = self.__pospq(red, q)
        re1, re2 = self.__size(re)
        core = y
        for qi in range(re2-1, -1, -1):
            if re[qi] in core:
                y = re
                break
            re = np.setdiff1d(re, re[qi])
            red = self.indisc(re, x)
            pos_red, _ = self.__pospq(red, q)
            if np.array_equal(pos_cd, pos_red):
                y = re
        y1, y2 = self.__size(y)
        j = 0
        for i in range(0, y2):
            y, j = redu2(j, y, c, d, x)
        return y

In [37]:
class RoughSetsSelector(BaseEstimator, SelectorMixin):

    def _get_support_mask(self):
        return self.mask_


    def fit(self, X, y=None):
        reducer = RoughSetsReducer()
        selected_ = reducer.reduce(X, y)
        B_unique_sorted, B_idx = np.unique(np.array(range(X.shape[1])), return_index=True)
        B_unique_sorted = B_unique_sorted + 1
        self.mask_ = np.in1d(B_unique_sorted, selected_, assume_unique=True)

        if self.mask_.size == 0:
            raise ValueError("No features were selected by rough sets reducer")
        return self

In [88]:
def cond_rules(y, X):
    for i in range(0, len(y)):
      print("Если размерности данных совпадают, то")
      print("-"*50)
      if len(y) == len(X[i]):
        print("Тогда можно продолжить вычисление редукции")
        print("-"*50)

      else:
        print("Нельзя продолжить вычисление редукции")
def approx_est(y, X_selected):
  '''
  Вычисление точности аппроксимации
  '''
  return mean_absolute_error(y, X_selected.reshape(-1,1)[0:len(y)]) / abs(y).sum() / 100 

In [38]:
y = np.array([[1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]]).T
X = np.array([[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1],
              [1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0],
              [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1],
              [1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1],
              [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],
              [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1],
              [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],
              [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1],
              [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1]])

selector = RoughSetsSelector()
X_selected = selector.fit(X, y).transform(X)

In [76]:
len(X[0])

12

In [39]:
X_selected

array([[1, 1],
       [0, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

In [36]:
result = pd.DataFrame(X_selected, columns=['upper', 'lower'])
result.to_csv("result.csv",index=False)

Введите датасет

In [ ]:
df = pd.read_csv("PATH_TO_DATASET")
y = df['target']
X = df.drop(['target'], axis=1)

Просмотр датасета

In [ ]:
df.head(5)

Применение алгоритма к датасету

In [ ]:
selector = RoughSetsSelector()
X_selected = selector.fit(X, y).transform(X)

In [89]:
cond_rules(y, X)

Если размерности данных совпадают, то
--------------------------------------------------
Тогда можно продолжить вычисление редукции
--------------------------------------------------
Если размерности данных совпадают, то
--------------------------------------------------
Тогда можно продолжить вычисление редукции
--------------------------------------------------
Если размерности данных совпадают, то
--------------------------------------------------
Тогда можно продолжить вычисление редукции
--------------------------------------------------
Если размерности данных совпадают, то
--------------------------------------------------
Тогда можно продолжить вычисление редукции
--------------------------------------------------
Если размерности данных совпадают, то
--------------------------------------------------
Тогда можно продолжить вычисление редукции
--------------------------------------------------
Если размерности данных совпадают, то
-----------------------------------------------

Вывод точности аппроксимации

In [74]:
np.round(approx_est(y, X_selected),4)

0.0006

Запись в .csv файл результата исполнения алгоритма

In [ ]:
result = pd.DataFrame(X_selected, columns=['upper', 'lower'])
result.to_csv("result.csv",index=False)

Пример работы

In [75]:
y = np.array([[1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]]).T
X = np.array([[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1],
              [1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0],
              [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1],
              [1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1],
              [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],
              [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1],
              [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],
              [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1],
              [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1]])

selector = RoughSetsSelector()
X_selected = selector.fit(X, y).transform(X)

In [ ]:
X_selected

In [ ]:
np.round(approx_est(y, X_selected),4)